In [1]:
# Importando os pacotes necessários
import pandas as pd
import numpy as np

from IPython.core.display import display, HTML

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
from sklearn.preprocessing import MinMaxScaler, StandardScaler

pd.set_option('display.max_columns', 10)
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def display_side_by_side(dfs: list, captions: list):
    """Display tables side by side to save vertical space
    Input:
        dfs: list of pandas.DataFrame
        captions: list of table captions
    """
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes(
            "style='display:inline'").set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0"
    display(HTML(output))

# Transformação de Dados
---

Em geral, após um processo de <ins>**Integração de dados**</ins>, colunas com diferentes tipos de dados podem ser geradas. 

Na <ins>**Transformação de dados**</ins>, o principal objetivo é exatamente **transformar** os dados de diferentes formatos em um formato suportado pelo processo de pesquisa, e.g., modelos e algoritmos. 

A <ins>**Transformação de dados**</ins> possui algumas etapas em comum com a <ins>**Limpeza de dados**</ins>, incluindo técnicas de remoção de ruído e discretização de dados. 

Portanto, nesta etapa, nós vamos investigar as demais etapas envolvidas nesta fase de *Preparação de dados*.

## Dados não padronizados

Dados coletados de diferentes fontes podem reunir dados **heterogêneos**, **não padronizados** e em **diferentes escalas**. 

O escalonamento de tais dados não é uma etapa obrigatória, mas uma boa prática! 

> **Mas... Por que precisamos dimensionar as variáveis em nosso conjunto de dados?**

> *Atributos padronizados podem auxiliar o desempenho de modelos, algoritmos e afins que são sensíveis ao dimensionamento dos atributos.*

Existem algumas técnicas de escalonamento que **transformam** características para uma escala, magnitude ou intervalo. 

Para descrever e exemplicar cada uma delas, utilizaremos o conjunto de atributos numéricos que descrevem as músicas do Spotify. Mais especificamente, utilizaremos apenas os atributos numéricos da tabela `Tracks` do nosso conjunto de dados. 

In [3]:
# Lendo os dados
data = pd.read_table('../dataset/spotify_hits_dataset_complete.tsv', encoding='utf-8')
data.head(3)

,song_id,song_name,artist_id,artist_name,popularity,...,liveness,loudness,speechiness,valence,tempo
0,2rRJrJEo19S2J82BDsQ3F7,Falling,['7uaIm6Pw7xplS8Dy06V6pT'],['Trevor Daniel'],77,...,0.0887,-8.756,0.0364,0.236,127.087
1,3BYIzNZ3t9lRQCACXSMLrT,Venetia,['4O15NlyKLIASxsJ0PrXPfz'],['Lil Uzi Vert'],66,...,0.1480,-4.139,0.1750,0.562,142.933
2,1g3J9W88hTG173ySZR6E9S,Tilidin Weg,['1aS5tqEs9ci5P9KD9tZWa6'],['Bonez MC'],13,...,0.0838,-8.544,0.2330,0.171,109.090


In [4]:
# Selecionando apenas as variáveis numéricas
df = data.select_dtypes(include=['number'])
print(df.columns.values) # imprime as colunas restantes

['popularity' 'track_number' 'num_artists' 'num_available_markets'
 'duration_ms' 'key' 'mode' 'time_signature' 'acousticness' 'danceability'
 'energy' 'instrumentalness' 'liveness' 'loudness' 'speechiness' 'valence'
 'tempo']


Depois de selecionar as variáveis numéricas da nossa tabela, ficamos com as seguintes informações:

* `popularity`: *score* de popularidade da track (o valor varia entre 0 e 100, sendo 100 o mais popular)
* `track_number`: número da track no um álbum que ela pertence
* `num_artists`: número total de intérpretes da track
* `num_available_markets`: número total de países nos quais a track pode ser reproduzida
* `duration_ms`: duração da track em milissegundos
* [`key` - `tempo`]: características acústicas da track

Utilizando esses atributos, nós iremos explorar dois tipos de escalonamento: *Normalização* e *Padronização*.

### NORMALIZAÇÃO

A *Normalização* é uma técnica de escalonamento em que os valores são **deslocados** e **redimensionados** para que fiquem entre 0 e 1. Essa técnica também é conhecida como nomarlização *Min-Max* e é usada para **transformar** dados em uma <ins>escala semelhante</ins>.

A fórmula da *Normalização* é dada por:

$$X' = \frac{X - X_{min}}{X_{max} - X_{min}},$$

onde $X_{max}$ e $X_{min}$ são os valores máximo e mínimo do atributo, respectivamente.

**OBSERVAÇÕES**
* Quando o valor de $X$ é o valor **mínimo** na coluna, o numerador será 0 e, portanto, $X'$ será igual a 0
* Por outro lado, quando o valor de $X$ é o valor **máximo** na coluna, o numerador é igual ao denominador e, portanto, o valor de $X'$ será igual a 1
* Se o valor de $X$ estiver entre o valor mínimo e máximo, então o valor de $X'$ estará entre 0 e 1

#### EXEMPLO
No exemplo a seguir, nós iremos normalizar todas as colunas numéricas da tabela `Tracks`.

Isto é, para cada atributo (i.e., coluna), o valor mínimo desse recurso será transformado em 0 e o valor máximo será transformado em 1, aplicando a fórmula descrita anteriormente da *Normalização Min-Max*. Para isso, nós utilizaremos os métodos `min()` e `max()` do *pandas*:

In [5]:
df_norm = df.copy()  # cópia do DataFrame

# Para cada coluna de df_norm,
for coluna in df_norm.columns:
    
    X = df_norm[[coluna]] # pegamos os valores da coluna
    X_ = (X - X.min()) / (X.max() - X.min()) # Normalização Min-Max 
    df_norm[[coluna]] = X_

display_side_by_side([df.head(3), df_norm.head(3)], ['Original', 'Normalizado']) # imprime as 3 primeiras linhas

,popularity,track_number,num_artists,num_available_markets,duration_ms,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo
0,77,10,1,177,159381,10,0,4,0.123000,0.784000,0.430000,0.000000,0.088700,-8.756000,0.036400,0.236000,127.087000
1,66,14,1,178,188800,9,1,4,0.162000,0.775000,0.757000,0.000000,0.148000,-4.139000,0.175000,0.562000,142.933000
2,13,1,1,0,180950,10,0,4,0.113000,0.882000,0.530000,0.479000,0.083800,-8.544000,0.233000,0.171000,109.090000
,popularity,track_number,num_artists,num_available_markets,duration_ms,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo
0,0.793814,0.310345,0.000000,0.994382,0.284679,0.909091,0.000000,0.750000,0.123557,0.769231,0.424552,0.000000,0.073225,0.615183,0.015335,0.212314,0.506887
1,0.680412,0.448276,0.000000,1.000000,0.349477,0.818182,1.000000,0.750000,0.162801,0.757692,0.773018,0.000000,0.136156,0.788266,0.176348,0.558386,0.606828
2,0.134021,0.000000,0.000000,0.000000,0.332187,0.909091,0.000000,0.750000,0.113495,0.894872,0.531117,0.502623,0.068025,0.623130,0.243727,0.143312,0.393380


#### OBSERVAÇÕES

Note que nós utilizamos `df_norm[[coluna]]` para selecionar as entradas de uma determinada coluna do nosso *DataFrame*, ao invés de apenas `df_norm[coluna]`.

> **Qual a diferença?**

Utilizando o método `type()` nós conseguimos verificar o tipo de classe do argumento passado como parâmetro:

In [6]:
type(df['popularity'])

pandas.core.series.Series

In [7]:
type(df[['popularity']])

pandas.core.frame.DataFrame

Portanto, quando utilizamos `[[]]`, é retornado um *DataFrame* formado apenas pelas colunas passadas por parâmetro. Já quando utilizamos `[]`, uma *Series* é retornada. Neste último caso, apenas uma coluna pode ser passada como parâmetro. Caso contrário, o seguinte erro será retornado:

In [9]:
df['popularity','track_number'].head(3)

KeyError: ('popularity', 'track_number')

In [10]:
df[['popularity','track_number']].head(3)

,popularity,track_number
0,77,10
1,66,14
2,13,1


#### EXEMPLO AVANÇADO

Uma outra forma de aplicar a *Normalização Min-Max* é utilizando o módulo `sklearn.preprocessing` e a classe `MinMaxScaler()` da biblioteca *sklearn*:

In [11]:
df_norm_2 = df_norm.copy()  # cópia do DataFrame
min_max_scaler = MinMaxScaler() # inicializa o transformador
df_norm_2.loc[:,] = min_max_scaler.fit_transform(df_norm_2) # Normalização Min-Max

display_side_by_side([df_norm.head(3), df_norm_2.head(3)], ['Normalização com pandas', 'Normalização com sklearn']) # imprime as 3 primeiras linhas

,popularity,track_number,num_artists,num_available_markets,duration_ms,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo
0,0.793814,0.310345,0.000000,0.994382,0.284679,0.909091,0.000000,0.750000,0.123557,0.769231,0.424552,0.000000,0.073225,0.615183,0.015335,0.212314,0.506887
1,0.680412,0.448276,0.000000,1.000000,0.349477,0.818182,1.000000,0.750000,0.162801,0.757692,0.773018,0.000000,0.136156,0.788266,0.176348,0.558386,0.606828
2,0.134021,0.000000,0.000000,0.000000,0.332187,0.909091,0.000000,0.750000,0.113495,0.894872,0.531117,0.502623,0.068025,0.623130,0.243727,0.143312,0.393380
,popularity,track_number,num_artists,num_available_markets,duration_ms,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo
0,0.793814,0.310345,0.000000,0.994382,0.284679,0.909091,0.000000,0.750000,0.123557,0.769231,0.424552,0.000000,0.073225,0.615183,0.015335,0.212314,0.506887
1,0.680412,0.448276,0.000000,1.000000,0.349477,0.818182,1.000000,0.750000,0.162801,0.757692,0.773018,0.000000,0.136156,0.788266,0.176348,0.558386,0.606828
2,0.134021,0.000000,0.000000,0.000000,0.332187,0.909091,0.000000,0.750000,0.113495,0.894872,0.531117,0.502623,0.068025,0.623130,0.243727,0.143312,0.393380


### PADRONIZAÇÃO

A *Padronização* é outra técnica de escalonamento em que os valores são **centralizados** em torno da média. Isso significa que a média de cada atributo se iguala a 0 e a distribuição resultante tem um desvio padrão igual a 1.

A fórmula da *Padronização* é dada por:

$$X' = \frac{X - \mu}{\sigma},$$

onde $\mu$ e $\sigma$ são a média e o desvio padrão dos valores do atributo, respectivamente.

#### EXEMPLO
No exemplo a seguir, nós iremos padronizar todas as colunas numéricas da tabela `Tracks`.

Isto é, para cada coluna, a média dos valores será subtraída pelo desvio padrão, aplicando a fórmula descrita anteriormente. Para isso, nós utilizaremos os métodos `mean()` e `std()` do *pandas*:

In [12]:
df_padron = df.copy()  # cópia do DataFrame

# Para cada coluna de df_padron,
for coluna in df_padron.columns:
    
    X = df_padron[[coluna]] # pegamos os valores da coluna
    X_ = (X - X.mean()) / (X.std()) # Padronização
    df_padron[[coluna]] = X_

display_side_by_side([df.head(3), df_padron.head(3)], ['Original', 'Padronizado']) # imprime as 3 primeiras linhas

,popularity,track_number,num_artists,num_available_markets,duration_ms,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo
0,77,10,1,177,159381,10,0,4,0.123000,0.784000,0.430000,0.000000,0.088700,-8.756000,0.036400,0.236000,127.087000
1,66,14,1,178,188800,9,1,4,0.162000,0.775000,0.757000,0.000000,0.148000,-4.139000,0.175000,0.562000,142.933000
2,13,1,1,0,180950,10,0,4,0.113000,0.882000,0.530000,0.479000,0.083800,-8.544000,0.233000,0.171000,109.090000
,popularity,track_number,num_artists,num_available_markets,duration_ms,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo
0,0.448125,1.007112,-0.644346,0.361651,-0.846963,1.277028,-1.138397,0.106209,-0.483188,0.621100,-1.231196,-0.160381,-0.640537,-0.896190,-0.802165,-1.217354,0.122480
1,-0.164943,1.806576,-0.644346,0.380038,-0.181157,1.003471,0.877744,0.106209,-0.327328,0.556130,0.809644,-0.160381,-0.225699,0.871711,0.462704,0.221431,0.658761
2,-3.118819,-0.791681,-0.644346,-2.892824,-0.358817,1.277028,-1.138397,0.106209,-0.523152,1.328550,-0.607086,6.303556,-0.674816,-0.815013,0.992015,-1.504228,-0.486598


#### EXEMPLO AVANÇADO

Uma outra forma de aplicar a *Padronização* é utilizando o módulo `sklearn.preprocessing` e a classe `StandardScaler()` da biblioteca *sklearn*:

In [13]:
df_padron_2 = df_padron.copy()  # cópia do DataFrame
standard_scaler = StandardScaler() # inicializa o transformador
df_padron_2.loc[:,] = standard_scaler.fit_transform(df_padron_2) # Padronização

display_side_by_side([df_padron.head(3), df_padron_2.head(3)], ['Padronização com pandas', 'Padronização com sklearn']) # imprime as 3 primeiras linhas

,popularity,track_number,num_artists,num_available_markets,duration_ms,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo
0,0.448125,1.007112,-0.644346,0.361651,-0.846963,1.277028,-1.138397,0.106209,-0.483188,0.621100,-1.231196,-0.160381,-0.640537,-0.896190,-0.802165,-1.217354,0.122480
1,-0.164943,1.806576,-0.644346,0.380038,-0.181157,1.003471,0.877744,0.106209,-0.327328,0.556130,0.809644,-0.160381,-0.225699,0.871711,0.462704,0.221431,0.658761
2,-3.118819,-0.791681,-0.644346,-2.892824,-0.358817,1.277028,-1.138397,0.106209,-0.523152,1.328550,-0.607086,6.303556,-0.674816,-0.815013,0.992015,-1.504228,-0.486598
,popularity,track_number,num_artists,num_available_markets,duration_ms,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo
0,0.448300,1.007505,-0.644597,0.361792,-0.847293,1.277526,-1.138841,0.106250,-0.483376,0.621342,-1.231676,-0.160443,-0.640787,-0.896539,-0.802477,-1.217828,0.122528
1,-0.165008,1.807280,-0.644597,0.380186,-0.181228,1.003862,0.878086,0.106250,-0.327456,0.556346,0.809959,-0.160443,-0.225787,0.872051,0.462885,0.221517,0.659018
2,-3.120034,-0.791989,-0.644597,-2.893951,-0.358957,1.277526,-1.138841,0.106250,-0.523356,1.329068,-0.607323,6.306012,-0.675079,-0.815330,0.992401,-1.504814,-0.486788


---
## Engenharia de características (Feature Engineering)

Outra forma de **transformar** os dados é através da **Engenharia de características**. Nesta técnica, características de um conjunto de dados podem ser criadas ou removidas para melhorar o desempenho de modelos de Aprendizado de Máquina. 

Muitas técnicas de <ins>**Transformação**</ins> e <ins>**Limpeza de dados**</ins> também podem ser consideradas **Engenharia de características**, incluindo métodos de escalonamento, discretização e redução de dimensionalidade. 

Portanto, a seguir, nós vamos discutir algumas técnicas que ainda não foram abordadas.

#### EXEMPLO

Para exemplificar, utilizaremos dessa vez as variáveis não-numéricas da tabela `Tracks`.

In [14]:
# Selecionando apenas variáveis não-numéricas
df = data.select_dtypes(exclude=['number'])
df = df.drop(columns=['song_id', 'song_name', 'artist_id', 'artist_name']) # removendo colunas desnecessárias
print(df.columns.values) # imprime as colunas restantes

['explicit' 'song_type' 'release_date']


Depois de selecionar as variáveis não-numéricas da nossa tabela e de remover colunas desnecessárias (i.e., ids e nomes), ficamos com os seguintes dados:

* `explicit`: *flag* indicando se a track contém conteúdo explícito
* `song_type`: se a track é do tipo *Solo* ou *Collaboration*
* `release_date`: data de lançamento da track

Utilizando esses atributos, nós iremos explorar duas formas de **Engenharia de características**: 

1. Transformando variáveis categóricas
2. Criar novos atributos a partir de variáveis de data

### DADOS CATEGÓRICOS

Variáveis categóricas são atributos que só podem assumir um número limitado e, geralmente, fixo de valores possíveis. 

Por exemplo, as músicas do Spotify contêm dados categóricos (`explicit`) para informar se as músicas são explícitas. 

Ou seja, cada unidade de observação (i.e., música) é atribuída a um determinado grupo ou categoria nominal (i.e. [`False` e `True`]). 

In [15]:
df[['explicit']].head(3)

,explicit
0,False
1,True
2,False


Muitos algoritmos e modelos de Aprendizado de Máquina têm dificuldades em processar dados categóricos, necessitando alguma forma de **transformar** tais atributos em variáveis numéricas. 

Uma das abordagens mais simples e comuns é converter variáveis categóricas em *dummies* ou indicadores, utilizando a função `get_dummies()` do *pandas*, conforme o exemplo a seguir. 

In [16]:
# Selecionando os atributos categóricos
df = data[['explicit', 'song_type']].astype('category')

# Convertendo em dummies
df_dummies = pd.get_dummies(df)

display_side_by_side([df.head(3), df_dummies.head(3)], ['Original', 'Transformado']) # imprime as 3 primeiras linhas

O método `get_dummies()` cria uma nova coluna contendo 0s e 1s para cada categoria possível do atributo, onde:

* `0`: indica que o dado não é daquela categoria
* `1`: indica que o dado é daquela categoria 

Como os dois atributos têm apenas duas categorias possíveis, o método criou duas colunas para cada atributo.

### VARIÁVEIS DE DATA

Datas e horários são fontes valiosas de informações que podem ser usadas em projetos de Ciência de Dados.

Por exemplo, em um modelo de predição de sucesso musical, pode ser útil considerar a data de lançamento das músicas. De fato, existem casos onde o período de lançamento influencia diretamente no sucesso musical.
**Exemplos:** músicas de Carnaval, músicas natalinas, músicas da Copa, etc.
 
No entanto, tecnicamente, as variáveis de data requerem alguma **Engenharia de características** para **transformá-las** em dados numéricos antes de serem usadas pelos algoritmos de Aprendizado de Máquina. 

#### EXEMPLO

Para exemplificar, utilizaremos a data de lançamento das músicas do Spotify (`release_date`) convertendo em formatos mais amigáveis, extraindo recursos e criando novas variáveis que podem ser usadas na análise de algum modelo. 

In [17]:
# Selecionando o id, nome e a data de lançamento das tracks
df = data[['song_id', 'song_name', 'release_date']]
df.head()

,song_id,song_name,release_date
0,2rRJrJEo19S2J82BDsQ3F7,Falling,2020-03-26
1,3BYIzNZ3t9lRQCACXSMLrT,Venetia,2020-03-06
2,1g3J9W88hTG173ySZR6E9S,Tilidin Weg,2020-07-30
3,75pQqzwgCjUOSSy5CpmAjy,Pero Ya No,2020-02-28
4,7kDUspsoYfLkWnZR7qwHZl,my ex's best friend (with blackbear),2020-09-25


O primeiro passo do processamento de datas é verificar se as variáveis estão em formato `datetime`.
Para isso, podemos utilizar o `dtypes` para verificar o tipo de de cada coluna do nosso *DataFrame*: 

In [18]:
df.dtypes # verificando o tipo das colunas de df

song_id         object
song_name       object
release_date    object
dtype: object

Como podemos notar, o tipo da variável `release_date` é `object` e não `datetime`. Portanto, precisamos convertê-la antes de passar para a **Engenharia de características**. Para isso, usaremos o método `to_datetime()` do *pandas*:

In [19]:
df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d') # convertendo a data de lançamento em 'datetime'
df.dtypes # verificando o tipo das colunas de df

song_id                 object
song_name               object
release_date    datetime64[ns]
dtype: object

Depois de converter para o formato correto, podemos gerar novas variáveis numéricas a partir da data de lançamento das músicas. Por exemplo, podemos quebrar a data em: dia, mês e ano: 

In [20]:
df['day'] = df['release_date'].dt.day  # dia
df['month'] = df['release_date'].dt.month  # mês
df['year'] = df['release_date'].dt.year  # ano
df.head(3)

,song_id,song_name,release_date,day,month,year
0,2rRJrJEo19S2J82BDsQ3F7,Falling,2020-03-26,26,3,2020
1,3BYIzNZ3t9lRQCACXSMLrT,Venetia,2020-03-06,6,3,2020
2,1g3J9W88hTG173ySZR6E9S,Tilidin Weg,2020-07-30,30,7,2020


Além dessas informações básicas, também podemos obter a semana do ano, o dia da semana e o trimestre:

In [21]:
df['week'] = df['release_date'].dt.week  # semana
df['dayofweek_index'] = df['release_date'].dt.dayofweek  # dia do mês (índice)
df['dayofweek_name'] = df['release_date'].dt.day_name() # dia do mês (nome)
df['quarter'] = df['release_date'].dt.quarter # trimestre
df.head(3)

,song_id,song_name,release_date,day,month,year,week,dayofweek_index,dayofweek_name,quarter
0,2rRJrJEo19S2J82BDsQ3F7,Falling,2020-03-26,26,3,2020,13,3,Thursday,1
1,3BYIzNZ3t9lRQCACXSMLrT,Venetia,2020-03-06,6,3,2020,10,4,Friday,1
2,1g3J9W88hTG173ySZR6E9S,Tilidin Weg,2020-07-30,30,7,2020,31,3,Thursday,3


Existem ainda outras informações possíveis de se obter como, por exemplo, hora, minutos, segundos, etc. 

---
## Dados desbalanceados

A última técnica de <ins>**Transformação de dados**</ins> que vamos analisar é a o **Balanceamento de dados**.  

Especificamente, dados balanceados são uma ocorrência comum em domínios reais, especialmente em modelos de classificação.

Podemos dizer que os dados estão desbalanceados quando existe uma **desproporção** de observações de cada classe do nosso modelo. 

Tal desbalanceamento não gera um erro imediato ao construir e executar um modelo. No entanto, os resultados podem ser ilusórios, dado que, por exemplo, a maioria das técnicas de Aprendizado de Máquina funcionam melhor quando o número de amostras em cada classe está equilibrado. 

### EXEMPLO

Para exemplificar, suponha que você tenha um modelo de classificação onde você precisa prever se uma música será uma colaboração ou não, com base em suas características acústicas. 

Resumindo, você teria um modelo de classificação binária onde cada música recebe o valor: 

* 1 se for uma colaboração, ou
* 0, caso contrário. 


Para começar, vamos selecionar apenas as colunas referentes aos nossos preditores (i.e., as características acústicas das músicas) e a nossa variável *target*, i.e., `song_type`:

In [22]:
cols = [
    'duration_ms', 'key', 'mode', 'time_signature', 'acousticness',
    'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness',
    'speechiness', 'valence', 'tempo', 'song_type'
]

# Selecionando algumas colunas da tabela Tracks
data = pd.read_table('../dataset/spotify_hits_dataset_complete.tsv', encoding='utf-8', usecols=cols)
data.head(3)

,song_type,duration_ms,key,mode,time_signature,...,liveness,loudness,speechiness,valence,tempo
0,Solo,159381,10,0,4,...,0.0887,-8.756,0.0364,0.236,127.087
1,Solo,188800,9,1,4,...,0.1480,-4.139,0.1750,0.562,142.933
2,Solo,180950,10,0,4,...,0.0838,-8.544,0.2330,0.171,109.090


Note que a variável *target* `song_type` está em formato categórico. Como vimos anteriormente, nós precisaríamos transformá-la em uma variável numérica caso esta variável fosse usada em algum algoritmo de Aprendizado de Máquina. 

No entanto, neste caso, tal variável corresponde ao que queremos prever (i.e., variável *target*). Portanto, não precisamos aplicar nenhum tratamento aqui.

Podemos, então, seguir para a verificação do balanceamento das classes (`Solo` e `Collaboration`). 
Para isso, usamos a função `value_counts()` que computa os valores únicos de cada categoria:

In [23]:
c = data['song_type'].value_counts() # quantidade de cada classe
p = data['song_type'].value_counts(normalize=True) # porcentagem de cada classe
pd.concat([c,round(p*100)], axis=1, keys=['counts', '%'])

,counts,%
Solo,752,59.0
Collaboration,532,41.0


Podemos observar que cerca de 41% das observações (i.e., 532 músicas) são colaborações. 

Portanto, se tivéssemos que prever sempre a classe 0, músicas `Solo`, obteríamos uma precisão de 59%.

A seguir, iremos testar esse cenário criando um modelo linear de regressão logística, usando o módulo `sklearn.linear_model` e a classe `LogisticRegression`. Nós seguimos os seguintes passos:

1. Separamos o conjunto de predidores (X) da nossa variável *target* (y);
2. Treinamos o modelo de regressão logística com os dois conjuntos X e y;
3. Realizamos a predição utilizando o mesmo conjunto de preditores X; e, por fim, 
4. Imprimimos os resultados.

In [24]:
# primeiro passo
y = data.song_type # variável target
X = data.drop('song_type', axis=1) # conjunto de preditores

# segundo passo
logr = LogisticRegression().fit(X, y) # treinando o modelo de regressão logística

# terceiro passo
pred_y_0 = logr.predict(X)
accuracy = round(accuracy_score(pred_y_0, y)*100)

# imprimindo os resultados
print(f'{accuracy}%') # acurácia do modelo
print(np.unique(pred_y_0)) # classes previstas pelo modelo

59%
['Solo']


Como podemos ver pelo resultado, nosso modelo apresentou uma precisão geral de 59%; ou seja, como consequência dos dados desbalanceados, o modelo prevê apenas uma classe. 

Em outras palavras, ele **ignora** a classe minoritária (i.e., `Collaborations`) em favor da classe majoritária (i.e., `Solo`). 

A seguir, aplicamos a técnica de *Downsampling*, que remove aleatoriamente observações da classe majoritária para evitar que seu sinal domine o algoritmo de aprendizagem. Para isso, seguimos alguns passos:

1. Separamos as observações de cada classe em diferentes *DataFrames*;
2. Realizamos uma amostragem da classe majoritária sem substituição para corresponder ao número total da classe minoritária; e, por fim,  
3. Concatenamos o *DataFrame* da classe minoritária com o *DataFrame* resultante.

In [25]:
# primeiro passo
df_majority = data[data.song_type == 'Solo'] # classe majoritária
df_minority = data[data.song_type == 'Collaboration'] # classe minoritária

# segundo passo
df_majority_downsampled = resample(
    df_majority, replace=False,  # amostra sem substituição
    n_samples=532,  # para corresponder à classe minoritária
    random_state=123)  # garantindo reprodutibilidade

# terceiro passo
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
df_downsampled.song_type.value_counts()

Solo             532
Collaboration    532
Name: song_type, dtype: int64

In [26]:
print(df_downsampled.shape[0], data.shape[0]) # comparando o tamanho dos DataFrames

1064 1284


Note que, apesar do novo *DataFrame* ter menos observações do que o original, a proporção das duas classes está **balanceada**. 

Agora, podemos treinar novamente o modelo de regressão logística! 

In [27]:
# primeiro passo
y = df_downsampled.song_type # variável target
X = df_downsampled.drop('song_type', axis=1) # conjunto de preditores

# segundo passo
logr = LogisticRegression().fit(X, y) # treinando o modelo de regressão logística

# terceiro passo
pred_y_1 = logr.predict(X)
accuracy = round(accuracy_score(pred_y_1, y)*100)

# imprimindo os resultados
print(f'{accuracy}%') # acurácia do modelo
print(np.unique(pred_y_1)) # classes previstas pelo modelo

49%
['Collaboration' 'Solo']


Obeserve que, após o balanceamento, o modelo consegue prever as duas classes.

Apesar de apresentar uma menor precisão (50%), o modelo apresenta uma métrica de avaliação mais significativa. 

## Conclusão

Este notebook apresentou como fazer a transformação de dados de diferentes formatos.

O próximo notebook ([4.3.Reducao.ipynb](4.3.Reducao.ipynb)) apresenta como aplicar técnicas de redução de dados para auxiliar a análise de dados com alta dimensionalidade.